In [1]:
!ml load cuda/11.8.0-fasrc01

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
from typing import Tuple

from nerfstudio.utils.eval_utils import eval_setup
from nerfstudio.cameras.rays import RayBundle, RaySamples
from nerfstudio.cameras.camera_optimizers import CameraOptimizer
from nerfstudio.cameras.cameras import Cameras
from nerfstudio.field_components.field_heads import FieldHeadNames

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Class implementation

In [3]:
config_path = "/n/home10/dpodolskyi/neural-registration/outputs/0_065_cat5_2/nerfacto/2024-08-01_153850/config.yml" 
_, pipeline, _, _ = eval_setup(config_path=Path(config_path), test_mode="inference")
nerf_model = pipeline.model
device = pipeline.device

AssertionError: Data directory data/datasets/0_065_cat5_2.0 does not exist.

In [ ]:
def create_ray_bundle(camera_params: torch.Tensor) -> RayBundle:
    # Unpack camera parameters
    position, rotation, fov = camera_params.split([3, 3, 1], dim=-1)

    # Create Cameras object
    cameras = Cameras(
        camera_to_worlds=torch.eye(4).unsqueeze(0).to(),
        fx=1.0,
        fy=1.0,
        cx=0.0,
        cy=0.0,
        width=self.pipeline.datamanager.train_dataset.cameras.width,
        height=self.pipeline.datamanager.train_dataset.cameras.height,
    )

    # Apply camera parameters
    cameras.position = position
    cameras.rotation = rotation
    cameras.fov = fov

    # Generate ray bundle
    ray_bundle = cameras.generate_rays(camera_indices=torch.zeros(1, device=self.device).long())

    return ray_bundle

def generate_rgb(self, camera_params: torch.Tensor) -> torch.Tensor:
    # Create a RayBundle from camera parameters
    ray_bundle = self.create_ray_bundle(camera_params)

    # Apply camera optimizer
    self.camera_optimizer.apply_to_raybundle(ray_bundle)

    # Get outputs from the model
    with torch.no_grad():
        outputs = self.model.get_outputs(ray_bundle)

    # Extract RGB values
    rgb = outputs[FieldHeadNames.RGB]

    return rgb

def optimize_camera(self, target_image: torch.Tensor, num_iterations: int = 1000) -> Tuple[
    torch.Tensor, torch.Tensor]:
    # Initialize camera parameters (position, rotation, fov)
    camera_params = torch.zeros(1, 7, device=self.device, requires_grad=True)

    for i in range(num_iterations):
        self.optimizer.zero_grad()

        # Generate RGB image from current camera parameters
        generated_rgb = self.generate_rgb(camera_params)

        # Compute loss
        loss = self.loss_fn(generated_rgb, target_image)

        # Backpropagate and optimize
        loss.backward()
        self.optimizer.step()

        if i % 100 == 0:
            print(f"Iteration {i}, Loss: {loss.item()}")

    # Return optimized camera parameters and final generated image
    return camera_params.detach(), self.generate_rgb(camera_params).detach()